In [2]:
import pygbif 
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import GPflow
import matplotlib.pyplot as plt
%matplotlib inline

# SDM with Gaussian Processes in Python
We're working to replicate and extend [@goldingn](http://github.com/goldingn)'s work in R. 

In [47]:
species = "Hyla walkeri"
speciesKey = pygbif.species.name_suggest(species)[0]['key']

In [48]:
searchArgs = {
    "taxonKey" : speciesKey,
    "limit" : 300,
    "fields" : 'minimal',
    "hasCoordinate" : True
}
dfs = [pd.DataFrame(pygbif.occurrences.search(offset=o, **searchArgs)['results'])
       for o in range(0, 1000, 300)]

In [55]:
data = pd.concat(dfs)
len(data)

463

In [50]:
np.any(pd.isnull(data['decimalLongitude'])) and pd.isnull(data['decimalLatitude'])

False

In [51]:
minLon, maxLon = data['decimalLongitude'].min(), data['decimalLongitude'].max()
minLat, maxLat = data['decimalLatitude'].min(), data['decimalLatitude'].max()
extent = (minLon, maxLon, minLat, maxLat)
extent

(-99.891666999999998,
 -87.617000000000004,
 15.310835406414707,
 20.091667000000001)

In [52]:
geometry = [Point(p) for p in zip(data.decimalLongitude, data.decimalLatitude)]
data = data.drop(['decimalLongitude', 'decimalLatitude'], axis=1)
geo_df = gpd.GeoDataFrame(data, geometry=geometry)

In [53]:
import folium
m = folium.Map( zoom_start=4)
json = folium.features.GeoJson(geo_df.to_json())
m.add_child(json)


In [87]:
from collections import Counter
Counter([item for sublist in geo_df.issues for item in sublist])

Counter({'COORDINATE_ROUNDED': 356,
         'GEODETIC_DATUM_ASSUMED_WGS84': 514,
         'MULTIMEDIA_DATE_INVALID': 9})

--- 
*a brief diversion to develop a mean function wrapper* 

In [8]:
class LambdaMeanFunction(GPflow.mean_functions.MeanFunction):
    """
   
    """
    def __init__(self, f=None):
        """
        f is a function whi
        ch is run on the data X and serves
        as the mean function.
        """
        
        self.f = Param(f)

    def __call__(self, X):
        return f(X)
    

In [111]:
from sklearn.linear_model import LinearRegression
